# Decision Trees
* Classifier
  + 카테고리 예측 (Yes/No, Class C1/C2/C3...)
* Tree 형태의 예측기
  + 대표적인 White-box 모델 (예측 과정과 이유를 투명하게 제공한다)
  + 모델을 이해하기 쉽다
* 참고: [wiki - 결정 트리](https://ko.wikipedia.org/wiki/%EA%B2%B0%EC%A0%95_%ED%8A%B8%EB%A6%AC)

# What is a Decision Tree
* 질문(판단) &rarr; 분기 &rarr; 하위 질문 &rarr; 분기 &rarr; ...
  + "스무고개" 퀴즈와 유사
* Example
  + "I am thinking of an animal"
  + <span style="color:blue">"Does it have more than five legs?"</span>
  + "No."
  + <span style="color:blue">"Is it delicious?"</span>
  + "No."
  + <span style="color:blue">"Does it appear on the back of the Australian five-cent coin?"</span>
  + "Yes"
  + <span style="color:blue">"Is it an enchidna?"</span>
  + "Yes, it is!"
* 학습을 통해 얻는 지식(모델)을 트리 형태로 관리
  + 트리를 따라 내려가면서 판단/분기 &rarr; 예측
  + Node: 분기 조건
  + Link: Yes/No
* Example Tree
  + <span style="color:blue">More than 5 legs?</span>
    - no
      + <span style="color:blue">Delicious?</span>
        - no
          + <span style="color:blue">On bank of Australian 5-cent coin?</span>
            - no
              + Kitty cat!
            - yes
              + Echidra
        - yes
          + <span style="color:blue">Star of <i>Charlotte's Web</i>?</span>
            - no
              + <span style="color:blue">Bison!</span>
            - yes
              + <span style="color:blue">Pig!</span>
    - yes
      + <span style="color:blue">Is hiding under your bed?</span>
        - no
          + <span style="color:blue">Makes honey?</span>
            - no
              + <span style="color:blue">Mosquito!</span>
            - yes
              + <span style="color:blue">Honeybee!</span>
        - yes
          + <span style="color:blue">Star of <i>Charlotte's Web</i>?</span>
            - no
              + <span style="color:blue">Bed bug!</span>
            - yes
              + <span style="color:blue">Spider!</span>

### 문제의 핵심
* "좋은 질문"을 해야한다!
* 질문의 순서가 중요하다!
  + 어떤 조건을 먼저 체크할 것인가?
  + 즉, 입력 데이터의 어떤 속성을 먼저 고려할 것인가?
* 각 분기 노드에서 속성을 선택하기 위한 합리적이고 견고한, 정량적인 판단 기준이 필요하다

# Entropy
* "좋은 질문"을 판단하기 위한 수단
  + choose questions whose answers give a lot of information what our tree should predict
  + **Entropy** 측정
    - Entropy가 높다 $\sim$ 불확실성이 크다 $\sim$ 판단의 기준(질문)으로 적합하지 않다
* Entropy = $\sum_{i=1}^C-p_i*\log_2(p_i)$
  + $p_i$: proportion of data labeled as class  $C_i$
* Entropy 특성
  + 예) 두 개의 Class $C_1$, $C_2$가 있을 때
    - $p_1$: 데이터 포인트가 $C_1$에 속할 확률
    - $p_2$: 데이터 포인트가 $C_2$에 속할 확률 (= $1-p_1$)
  + $p_1=0$ ($p_2=1$) 또는 $p_1=1$ ($p_2=0$)
    - Entropy=0
    - 불확실성이 없다
  + $p_1=p_2=0.5$
    - Entropy=1
    - 불확실성 최대
* 참고: [Entory, Gini, and Information Gain](https://www.bogotobogo.com/python/scikit-learn/scikt_machine_learning_Decision_Tree_Learning_Informatioin_Gain_IG_Impurity_Entropy_Gini_Classification_Error.php)

In [1]:
from matplotlib import pyplot as plt
import math
import numpy as np

# C1, C2 두 개의 클래스(카테고리)가 있을 때
# p1의 값에 따라 엔트로피가 어떻게 달라지는가 그려보자
x = [f for f in np.arange(0.01,1, 0.01)]
y1 = [-xi*math.log(xi,2) for xi in x]
y2 = [-xi*math.log(xi,2) -(1-xi)*math.log((1-xi),2) for xi in x]

plt.plot(x,y1, label='-p*log(p)')
plt.plot(x,y2, label='sum(-pi*log(pi))')
plt.xlabel('p')
plt.legend()
plt.show()

<Figure size 640x480 with 1 Axes>

## Entropy 구해보기 - 연습
### 누가 군대에 갈까요?
* 전체 100명
  + (성별) 남자 : 50, 여자 : 50
  + (몸무게)  60Kg 이상 : 60,  60Kg 이하 : 40
* 100명 중 군대간 사람은 40명
  + 40명 모두 남자
  + 40명 중 30명은 몸무게 60Kg 이상
* "성별=='남'"일 때
  + 군대간 사람: 40, 안간사람:10
  + 군대갈 확률 : 40/50
  + 군대 안갈 확률 : 10/50
  + entropy([0.8, 0.2]) = $-0.8log_2{0.8} -0.2log_2{0.2}$ = 0.72
* "몸무게=='60Kg이상'"일 때
  + 군대간 사람: 30, 안간사람:30
  + 군대갈 확률 : 30/60
  + 군대 안갈 확률 : 30/60
  + entropy([0.5, 0.5]) = $-0.5log_2{0.5} -0.5log_2{0.5}$ = 1
* 어떤걸  먼저 물어볼까?
  + "**성별**" vs. "몸무게"

In [2]:
from typing import List
import math

def entropy(class_probabilities: List[float]) -> float:
    """Given a list of class probabilities, compute the entropy"""
    return sum(-p * math.log(p, 2)
               for p in class_probabilities
               if p > 0)                     # ignore zero probabilities

assert entropy([1.0]) == 0
assert entropy([0.5, 0.5]) == 1
assert 0.81 < entropy([0.25, 0.75]) < 0.82

In [3]:
entropy([.8,.2])

0.7219280948873623

In [4]:
entropy([.5,.5])

1.0

### Data Entropy 구하기
* 주어진 데이터셋의 엔트로피 구하기
    + 훈련용 데이터셋의 부분집합 (예, "성별=남자"인 데이터셋)
* 각 Class(Label,카테고리)에 속할 확률을 구하면 해당 데이터셋의 Entropy를 계산할 수 있다
    + 예) "성별=남자"인 데이터셋에서 "군대에 갈 확률", "군대에 안 갈 확률" 구하기

In [5]:
from typing import Any
from collections import Counter

def class_probabilities(labels: List[Any]) -> List[float]:
    total_count = len(labels)
    return [count / total_count
            for count in Counter(labels).values()]

def data_entropy(labels: List[Any]) -> float:
    return entropy(class_probabilities(labels))

assert data_entropy(['a']) == 0
assert data_entropy([True, False]) == 1
assert data_entropy([3, 4, 4, 4]) == entropy([0.25, 0.75])

# The Entropy of a Partition
* 어떤 Feature(질문)을 선택할 것인가 판단하기 위함
* Partition $\sim$ Subtree
  + 예) "성별"을 가지고 나눈다면
    - Subsets : 남자, 여자
* $H=q_1H(S_1) + q_2H(S_2) +...+q_mH(S_m)$
  + $H$: Partition Entory
    - 예) "성별"을 기준으로 했을 때의 Entropy
  + $H(S_i)$: Data Entropy
    - 예) "성별=남자"인 데이터셋의 Entropy
  + $q_i$: 해당 Partion($S$)에서 데이터셋($S_i$)의 비율
    - 예) "성별=남자"의 비율

In [6]:
def partition_entropy(subsets: List[List[Any]]) -> float:
    """Returns the entropy from this partition of data into subsets"""
    total_count = sum(len(subset) for subset in subsets)

    return sum(data_entropy(subset) * len(subset) / total_count
               for subset in subsets)

# Creating a Decision Tree
### 서류전형 자동 분류기
* Example Dataset
  + 채용 인터뷰 데이터
  + Features
    - 지원자 등급(level), 주로 사용하는 언어(lang), 트위터 사용여부(tweets), 박사학위 여부(phd), 인터뷰결과(did_well)
    - 인터뷰결과(did_well): 예측하고자 하는 값(Class/Category/Label, 종속변수, Dependent Variable)
* 알고리즘
  1. 각 Feature에 대해 Partition Entropy를 구한다
  2. Partition Entropy가 가장 작은 Feature를 선택한다. (Node)
  3. 선택된 Feature의 값을 기준으로 분기한다. (Edge)
  4. 분기된 각 데이터셋(Subset)에 대해 1~3 과정을 반복하면서 Subtree를 만들어 나간다
     + 이 때, 먼저 선택한 Feature는 제외한다
  5. 반복 과정에서 분기된 데이터셋이 모든 같은 Label을 가지면 더 이상 분기하지 않는다
* 참고: [ID3 Algorithm](https://en.wikipedia.org/wiki/ID3_algorithm)
  

In [7]:
from typing import Dict, TypeVar
from collections import defaultdict

T = TypeVar('T')  # generic type for inputs

def partition_by(inputs: List[T], attribute: str) -> Dict[Any, List[T]]:
    """Partition the inputs into lists based on the specified attribute."""
    partitions: Dict[Any, List[T]] = defaultdict(list)
    for input in inputs:
        key = getattr(input, attribute)  # value of the specified attribute
        partitions[key].append(input)    # add input to the correct partition
    return partitions

def partition_entropy_by(inputs: List[Any],
                         attribute: str,
                         label_attribute: str) -> float:
    """Compute the entropy corresponding to the given partition"""
    # partitions consist of our inputs
    partitions = partition_by(inputs, attribute)

    # but partition_entropy needs just the class labels
    labels = [[getattr(input, label_attribute) for input in partition]
              for partition in partitions.values()]

    return partition_entropy(labels)

In [8]:
from typing import NamedTuple, Optional

class Candidate(NamedTuple):
    level: str
    lang: str
    tweets: bool
    phd: bool
    did_well: Optional[bool] = None  # allow unlabeled data

                  #  level     lang     tweets  phd  did_well
inputs = [Candidate('Senior', 'Java',   False, False, False),
          Candidate('Senior', 'Java',   False, True,  False),
          Candidate('Mid',    'Python', False, False, True),
          Candidate('Junior', 'Python', False, False, True),
          Candidate('Junior', 'R',      True,  False, True),
          Candidate('Junior', 'R',      True,  True,  False),
          Candidate('Mid',    'R',      True,  True,  True),
          Candidate('Senior', 'Python', False, False, False),
          Candidate('Senior', 'R',      True,  False, True),
          Candidate('Junior', 'Python', True,  False, True),
          Candidate('Senior', 'Python', True,  True,  True),
          Candidate('Mid',    'Python', False, True,  True),
          Candidate('Mid',    'Java',   True,  False, True),
          Candidate('Junior', 'Python', False, True,  False)
         ]

## Decision Tree - 손으로 그려보자
* 어떤 Feature를 먼저 선택할 것인가?
  + 각 Feature에 대해 Partition Entropy를 구한다

In [9]:
for key in ['level','lang','tweets','phd']:
    print(key, partition_entropy_by(inputs, key, 'did_well'))

level 0.6935361388961919
lang 0.8601317128547441
tweets 0.7884504573082896
phd 0.8921589282623617


### (node)'level'
* 가장 작은 Partition Entropy: 'level'

In [10]:
partitions = partition_by(inputs, 'level')
partitions.values()

dict_values([[Candidate(level='Senior', lang='Java', tweets=False, phd=False, did_well=False), Candidate(level='Senior', lang='Java', tweets=False, phd=True, did_well=False), Candidate(level='Senior', lang='Python', tweets=False, phd=False, did_well=False), Candidate(level='Senior', lang='R', tweets=True, phd=False, did_well=True), Candidate(level='Senior', lang='Python', tweets=True, phd=True, did_well=True)], [Candidate(level='Mid', lang='Python', tweets=False, phd=False, did_well=True), Candidate(level='Mid', lang='R', tweets=True, phd=True, did_well=True), Candidate(level='Mid', lang='Python', tweets=False, phd=True, did_well=True), Candidate(level='Mid', lang='Java', tweets=True, phd=False, did_well=True)], [Candidate(level='Junior', lang='Python', tweets=False, phd=False, did_well=True), Candidate(level='Junior', lang='R', tweets=True, phd=False, did_well=True), Candidate(level='Junior', lang='R', tweets=True, phd=True, did_well=False), Candidate(level='Junior', lang='Python', tw

In [11]:
by_level = [(i.level, i.did_well) for i in inputs]
by_level.sort(key=lambda x: x[0])
by_level

[('Junior', True),
 ('Junior', True),
 ('Junior', False),
 ('Junior', True),
 ('Junior', False),
 ('Mid', True),
 ('Mid', True),
 ('Mid', True),
 ('Mid', True),
 ('Senior', False),
 ('Senior', False),
 ('Senior', False),
 ('Senior', True),
 ('Senior', True)]

* 'level' 을 기준으로 분기했을 때
  + 'Junior': True 3, False 2
  + 'Mid': True 4, False 0 (모두 같은  클래스에 속하므로 더 이상 분기하지 않는다)
  + 'Senior': True 2, False 3
* 만약 'level'만 가지고 인터뷰결과를 예측한다면?
  + IF 'Junior' THEN True, ELSE IF 'Mid' THEN True, ELSE False
* 'Junior'와 'Senior'는 한 단계 더 내려가 보면 좋겠다
  + 먼저 'Senior'에 대해 한 단계 내려가 보자

### (node)'level' &rarr; (edge)level='Senior'

In [12]:
senior_inputs = [input for input in inputs if input.level == 'Senior']

* 나머지 Features 중에서 어떤 Feature를 선택할 것인가?

In [13]:
for key in ['lang','tweets','phd']: # 'level' 제외
    print(key, partition_entropy_by(senior_inputs, key, 'did_well'))

lang 0.4
tweets 0.0
phd 0.9509775004326938


### (node)'level' &rarr; (edge)level='Senior' &rarr;  (node)'tweets'
* 가장 작은 Partition Entropy: 'tweets'
* 'tweets' 값을 기준으로 분기

In [14]:
by_tweets = [(i.tweets, i.did_well) for i in senior_inputs]
by_tweets.sort(key=lambda x: x[0])
by_tweets

[(False, False), (False, False), (False, False), (True, True), (True, True)]

* 'tweets'를 기준으로 분기했을 때
  + False : True 0, False 3
  + True : True 2, False 0
  + 두 경우 모두 같은 클래스에 속하므로 더 이상 분기하지 않는다

### (node)'level' &rarr; (edge)level='Junior'
* 'Junior'에 대해서도 위와 같은 방식으로 Subtree를 그릴 수 있다

### 최종 결과 (Decision Tree)
* **level?**
  + <span style="color:blue">Senior</span>
    - **tweets?**
      + <span style="color:blue">Yes</span>
        - **HIRE!**
      + <span style="color:blue">No</span>
        - **DO NOT HIRE!**
  + <span style="color:blue">Mid</span>
    - **HIRE!**
  + <span style="color:blue">Junior</span>
    - **phd?**
      + <span style="color:blue">No</span>
        - **HIRE!**
      + <span style="color:blue">Yes</span>
        - **DO NOT HIRE!**

# Putting It All Together
* 학습
  + Training Dataset을 이용하여 Decision Tree 만들기
* 예측
  + Decision Tree 탐색하기

### Tree 자료 구조

In [15]:
from typing import NamedTuple, Union, Any

class Leaf(NamedTuple):
    value: Any

class Split(NamedTuple):
    attribute: str
    subtrees: dict
    default_value: Any = None

DecisionTree = Union[Leaf, Split]

위와 같은 클래스로 앞에서 구한 서류전형 자동 분류기 트리를 표현하면 다음과 같다

In [16]:
hiring_tree = Split('level', {   # First, consider "level".
    'Junior': Split('phd', {     # if level is "Junior", next look at "phd"
        False: Leaf(True),       #   if "phd" is False, predict True
        True: Leaf(False)        #   if "phd" is True, predict False
    }),
    'Mid': Leaf(True),           # if level is "Mid", just predict True
    'Senior': Split('tweets', {  # if level is "Senior", look at "tweets"
        False: Leaf(False),      #   if "tweets" is False, predict False
        True: Leaf(True)         #   if "tweets" is True, predict True
    })
})

### Classifier
* Decision Tree를 이용한 예측기

In [17]:
def classify(tree: DecisionTree, input: Any) -> Any:
    """classify the input using the given decision tree"""

    # If this is a leaf node, return its value
    if isinstance(tree, Leaf):
        return tree.value

    # Otherwise this tree consists of an attribute to split on
    # and a dictionary whose keys are values of that attribute
    # and whose values of are subtrees to consider next
    subtree_key = getattr(input, tree.attribute)

    if subtree_key not in tree.subtrees:   # If no subtree for key,
        return tree.default_value          # return the default value.

    subtree = tree.subtrees[subtree_key]   # Choose the appropriate subtree
    return classify(subtree, input)        # and use it to classify the input.

### Trainer
* Decision Tree 만들기

In [18]:
def build_tree_id3(inputs: List[Any],
                   split_attributes: List[str],
                   target_attribute: str) -> DecisionTree:
    # Count target labels
    label_counts = Counter(getattr(input, target_attribute)
                           for input in inputs)
    most_common_label = label_counts.most_common(1)[0][0]

    # If there's a unique label, predict it
    if len(label_counts) == 1:
        return Leaf(most_common_label)

    # If no split attributes left, return the majority label
    if not split_attributes:
        return Leaf(most_common_label)

    # Otherwise split by the best attribute

    def split_entropy(attribute: str) -> float:
        """Helper function for finding the best attribute"""
        return partition_entropy_by(inputs, attribute, target_attribute)

    best_attribute = min(split_attributes, key=split_entropy)

    partitions = partition_by(inputs, best_attribute)
    new_attributes = [a for a in split_attributes if a != best_attribute]

    # recursively build the subtrees
    subtrees = {attribute_value : build_tree_id3(subset,
                                                 new_attributes,
                                                 target_attribute)
                for attribute_value, subset in partitions.items()}

    return Split(best_attribute, subtrees, default_value=most_common_label)


In [19]:
tree = build_tree_id3(inputs,
                      ['level', 'lang', 'tweets', 'phd'],
                      'did_well')

# Should predict True
assert classify(tree, Candidate("Junior", "Java", True, False))

# Should predict False
assert not classify(tree, Candidate("Junior", "Java", True, True))

# Should predict True
assert classify(tree, Candidate("Intern", "Java", True, True))

# Random Forests
* Decision Tree를 여러개 만든다
  + 예) Training Dataset을 각각 다르게 구성하여 학습
  + 참고: bootstrap_sample(inputs)
* 여러 Decision Tree의 결과 → Voting

In [20]:
# 'scratch' package가 설치된 디렉토리로 작업 디렉토리 바꿈 (package를 import하기 위함)
import os, sys
os.chdir('..')

from scratch.multiple_regression import bootstrap_sample

<Figure size 432x288 with 0 Axes>

In [24]:
def forest_classify(trees, input):
    votes = [classify(tree, input) for tree in trees]
    vote_counts = Counter(votes)
    return vote_counts.most_common(1)[0][0]


def build_forest(inputs, num_trees=3, num_samples=10):
    trees = []
    for i in range(num_trees):
        #tree = build_tree_id3(random.sample(inputs, num_samples))
        tree = build_tree_id3(bootstrap_sample(inputs), ['level', 'lang', 'tweets', 'phd'], 'did_well')
        trees.append(tree)
    
    return trees

In [25]:
trees = build_forest(inputs, num_samples=5)

In [27]:
print("Junior / Java / tweets / phd ==>", classify(tree, Candidate("Junior", "Java", True, False)))

Junior / Java / tweets / phd ==> True


In [29]:
print("Junior / Java / tweets / phd ==>", forest_classify(trees, Candidate("Junior", "Java", True, False)))

Junior / Java / tweets / phd ==> True


# For Further Exploration
* ID3
  + Categorical features
* C4.5
  + Categorical & Numerical
* CART
  + Categorical & Numerical, Classification & Regression
  + *scikit-learn uses an optimised version of the CART algorithm*
  
scikit-learn: [Decision Trees](https://scikit-learn.org/stable/modules/tree.html)